# See notes below for blockage on implementing monitoring and frontend monintoring

In [67]:
import hopsworks
import src.config as config
# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()


2025-01-25 18:20:19,599 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-25 18:20:19,607 INFO: Initializing external client
2025-01-25 18:20:19,607 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-25 18:20:20,622 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104


# Checking what is created in 12-feature-pipeline for "Time-series data at hourly frequency"

In [68]:
# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,# "time_series_hourly_feature_group"
    version=3,# config.FEATURE_GROUP_VERSION needs updated in config
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)

In [69]:
a = feature_group.read()
a.columns


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.63s) 


Index(['pickup_hour', 'rides', 'pickup_location_id', 'pickup_ts'], dtype='object')

## Produces output as expected 

In [70]:
a

,pickup_hour,rides,pickup_location_id,pickup_ts
0,2024-10-03 21:00:00+00:00,0,60,1727989200000
1,2024-10-25 13:00:00+00:00,0,174,1729861200000
2,2024-10-03 13:00:00+00:00,0,51,1727960400000
3,2024-10-19 13:00:00+00:00,131,90,1729342800000
4,2024-10-17 17:00:00+00:00,0,117,1729184400000
...,...,...,...,...
171610,2024-10-15 04:00:00+00:00,0,81,1728964800000
171611,2024-10-08 23:00:00+00:00,94,68,1728428400000
171612,2024-10-06 00:00:00+00:00,0,251,1728172800000
171613,2024-10-29 16:00:00+00:00,0,202,1730217600000


## Running the full 28 days as we expect :)

In [71]:
print(a.pickup_hour.min())
print(a.pickup_hour.max())

2024-10-02 23:00:00+00:00
2024-10-30 23:00:00+00:00


# Checking 14-inference-pipeline

In [72]:
from src.feature_store_api import get_feature_store
# connect to the feature group
feature_group_b = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=4,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

2025-01-25 18:20:32,734 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-25 18:20:32,738 INFO: Initializing external client
2025-01-25 18:20:32,738 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-25 18:20:33,786 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104


In [73]:
b = feature_group_b.read()
b.columns

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.63s) 


Index(['pickup_location_id', 'predicted_demand', 'pickup_hour'], dtype='object')

In [74]:
b

,pickup_location_id,predicted_demand,pickup_hour
0,239,9.0,2024-10-31 08:00:00+00:00
1,95,9.0,2024-10-31 08:00:00+00:00
2,46,9.0,2024-10-31 08:00:00+00:00
3,241,9.0,2024-10-31 08:00:00+00:00
4,143,10.0,2024-10-31 08:00:00+00:00
...,...,...,...
250,112,9.0,2024-10-31 08:00:00+00:00
251,208,10.0,2024-10-31 08:00:00+00:00
252,197,10.0,2024-10-31 08:00:00+00:00
253,40,11.0,2024-10-31 08:00:00+00:00


In [75]:
b

,pickup_location_id,predicted_demand,pickup_hour
0,239,9.0,2024-10-31 08:00:00+00:00
1,95,9.0,2024-10-31 08:00:00+00:00
2,46,9.0,2024-10-31 08:00:00+00:00
3,241,9.0,2024-10-31 08:00:00+00:00
4,143,10.0,2024-10-31 08:00:00+00:00
...,...,...,...
250,112,9.0,2024-10-31 08:00:00+00:00
251,208,10.0,2024-10-31 08:00:00+00:00
252,197,10.0,2024-10-31 08:00:00+00:00
253,40,11.0,2024-10-31 08:00:00+00:00


In [76]:
print(b.pickup_hour.min())
print(b.pickup_hour.max())

2024-10-31 08:00:00+00:00
2024-10-31 08:00:00+00:00


In [62]:
# So setting the current_date to a fixed date means there are 2 things that need to happen:

# 1: Every hour this runs we need to simulate +1 hour in time. Meaning current time + 1 hour 
# 2: Surely we want to start from here '2024-10-02 23:00:00+00:00' as we have known data points from here to '2024-10-30 23:00:00+00:00'

In [63]:
# To check

# model_predictions_feature_group vs time_series_hourly_feature_group

## model_predictions_feature_group from 14-inference pipeline 

In [77]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
model_predictions_feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=3,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

2025-01-25 18:21:02,956 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-25 18:21:02,959 INFO: Initializing external client
2025-01-25 18:21:02,959 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-25 18:21:03,984 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104


In [78]:
b = model_predictions_feature_group.read()
b.columns
print(b.pickup_hour.min())
print(b.pickup_hour.max())
b

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.54s) 
2024-10-31 03:00:00+00:00
2024-10-31 03:00:00+00:00


,pickup_location_id,predicted_demand,pickup_hour,pickup_ts
0,48,6.0,2024-10-31 03:00:00+00:00,1730329
1,215,10.0,2024-10-31 03:00:00+00:00,1730329
2,150,9.0,2024-10-31 03:00:00+00:00,1730329
3,143,10.0,2024-10-31 03:00:00+00:00,1730329
4,15,9.0,2024-10-31 03:00:00+00:00,1730329
...,...,...,...,...
250,229,8.0,2024-10-31 03:00:00+00:00,1730329
251,169,9.0,2024-10-31 03:00:00+00:00,1730329
252,12,9.0,2024-10-31 03:00:00+00:00,1730329
253,264,9.0,2024-10-31 03:00:00+00:00,1730329


In [79]:
# Show unique hours
unique_hours = b['pickup_hour'].unique()
print("Unique pickup hours in dataset:")
print(sorted(unique_hours))

# Optional: Show count per hour
hour_counts = b['pickup_hour'].value_counts().sort_index()
print("\nCounts per hour:")
print(hour_counts)

Unique pickup hours in dataset:
[Timestamp('2024-10-31 03:00:00+0000', tz='Etc/UTC')]

Counts per hour:
pickup_hour
2024-10-31 03:00:00+00:00    255
Name: count, dtype: int64


## time_series_hourly_feature_group

In [82]:
# connect to the feature group
time_series_feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,# "time_series_hourly_feature_group"
    version=3,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)

In [83]:
c = time_series_feature_group.read()
c.sort_values('pickup_hour')

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.27s) 


,pickup_hour,rides,pickup_location_id,pickup_ts
48937,2024-10-02 23:00:00+00:00,10,88,1727910000000
48921,2024-10-02 23:00:00+00:00,7,145,1727910000000
48922,2024-10-02 23:00:00+00:00,0,228,1727910000000
48923,2024-10-02 23:00:00+00:00,0,241,1727910000000
48827,2024-10-02 23:00:00+00:00,16,261,1727910000000
...,...,...,...,...
33789,2024-10-30 23:00:00+00:00,171,163,1730329200000
108861,2024-10-30 23:00:00+00:00,0,174,1730329200000
154122,2024-10-30 23:00:00+00:00,120,164,1730329200000
114855,2024-10-30 23:00:00+00:00,0,227,1730329200000


In [84]:
c.close

AttributeError: 'DataFrame' object has no attribute 'close'

In [85]:
# Show unique hours
unique_hours = c['pickup_hour'].unique()
print("Unique pickup hours in dataset:")
print(sorted(unique_hours))

# Optional: Show count per hour
hour_counts = c['pickup_hour'].value_counts().sort_index()
print("\nCounts per hour:")
print(hour_counts)

Unique pickup hours in dataset:
[Timestamp('2024-10-02 23:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 00:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 01:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 02:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 03:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 04:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 05:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 06:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 07:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 08:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 09:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 10:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 11:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 12:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 13:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 14:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 15:00:00+0000', tz='Etc/UTC'), Timestamp('2024-10-03 16:00:00+0000', tz='Etc/UTC'), Timestamp('20

In [86]:
c.columns

Index(['pickup_hour', 'rides', 'pickup_location_id', 'pickup_ts'], dtype='object')

In [88]:
b

,pickup_location_id,predicted_demand,pickup_hour,pickup_ts
0,48,6.0,2024-10-31 03:00:00+00:00,1730329
1,215,10.0,2024-10-31 03:00:00+00:00,1730329
2,150,9.0,2024-10-31 03:00:00+00:00,1730329
3,143,10.0,2024-10-31 03:00:00+00:00,1730329
4,15,9.0,2024-10-31 03:00:00+00:00,1730329
...,...,...,...,...
250,229,8.0,2024-10-31 03:00:00+00:00,1730329
251,169,9.0,2024-10-31 03:00:00+00:00,1730329
252,12,9.0,2024-10-31 03:00:00+00:00,1730329
253,264,9.0,2024-10-31 03:00:00+00:00,1730329


In [91]:
c = c.sort_values('pickup_hour')

In [92]:
# Check unique combinations before merging
print("Unique location-hour pairs in df_A:", b.groupby(['pickup_location_id', 'pickup_hour']).size().shape[0])
print("Unique location-hour pairs in df_B:", c.groupby(['pickup_location_id', 'pickup_hour']).size().shape[0])

# Merge and check results
comparison_df = b.merge(c, on=['pickup_location_id', 'pickup_hour'], how='inner')
print("\nShape after merge:", comparison_df.shape)

# Calculate prediction error
comparison_df['prediction_error'] = comparison_df['predicted_demand'] - comparison_df['rides']

comparison_df

Unique location-hour pairs in df_A: 255
Unique location-hour pairs in df_B: 171615

Shape after merge: (0, 6)


,pickup_location_id,predicted_demand,pickup_hour,pickup_ts_x,rides,pickup_ts_y,prediction_error


In [93]:
# Show unique hours
unique_hours = comparison_df['pickup_hour'].unique()
print("Unique pickup hours in dataset:")
print(sorted(unique_hours))

# Optional: Show count per hour
hour_counts = comparison_df['pickup_hour'].value_counts().sort_index()
print("\nCounts per hour:")
print(hour_counts)

Unique pickup hours in dataset:
[]

Counts per hour:
Series([], Name: count, dtype: int64)


In [94]:
comparison_df['prediction_error'].mean()

nan

# Checking the issues with Monitoring pipeline

In [95]:
import hopsworks
import src.config as config
project = hopsworks.login(
        project=config.HOPSWORKS_PROJECT_NAME,
        api_key_value=config.HOPSWORKS_API_KEY
)

2025-01-25 18:23:28,405 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-25 18:23:28,408 INFO: Initializing external client
2025-01-25 18:23:28,408 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-25 18:23:29,448 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104


# Plan is to:

* Create a new feature group for  'time_series_hourly_feature_group primary key' 
* Just make it version V4 and keep it separate as a test. You will need to rerun some things for this
* You can then isolate the issue
* The priority will be to make the 'time_series_hourly_feature_group primary key 'pickup_hour'
* Hopefully this will fix the issues with query when creating a 'monitoring feature view' 
* Then see where you are at 

In [96]:
fs = project.get_feature_store()
actuals_fg = fs.get_feature_group(name ='time_series_hourly_feature_group', version = 3)
print(actuals_fg.schema)

[Feature('pickup_hour', 'timestamp', None, False, False, None, None, 1393062), Feature('rides', 'bigint', None, False, False, None, None, 1393062), Feature('pickup_location_id', 'int', None, True, False, None, None, 1393062), Feature('pickup_ts', 'bigint', None, True, False, None, None, 1393062)]


In [99]:
actuals_df = actuals_fg.read()
actuals_df.head(5)
actuals_df = actuals_df.sort_values('pickup_hour')


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (10.37s) 


In [100]:
predictions_fg = fs.get_feature_group(name ='model_predictions_feature_group', version =4)
print(predictions_fg.schema)
 

predictions_df =predictions_fg.read()
predictions_df.head(5)

[Feature('pickup_location_id', 'int', None, True, False, None, None, 1399239), Feature('predicted_demand', 'double', None, False, False, None, None, 1399239), Feature('pickup_hour', 'timestamp', None, True, False, None, None, 1399239)]
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.51s) 


,pickup_location_id,predicted_demand,pickup_hour
0,239,9.0,2024-10-31 08:00:00+00:00
1,95,9.0,2024-10-31 08:00:00+00:00
2,46,9.0,2024-10-31 08:00:00+00:00
3,241,9.0,2024-10-31 08:00:00+00:00
4,143,10.0,2024-10-31 08:00:00+00:00


In [101]:
import pandas as pd
from datetime import datetime, timedelta,timezone
current_date = pd.Timestamp('2024-10-30 23:00:00')
from_date = current_date - timedelta(days=14)
to_date=current_date
# from_ts = int(from_date.timestamp() * 1000)
# to_ts = int(to_date.timestamp() * 1000)
# # from_date_ = pd.Timestamp(current_date - timedelta(days=28)).replace(tzinfo=None)
# print(from_date)
# # to_date_ = pd.Timestamp(from_date_ + timedelta(hours=1)).replace(tzinfo=None)
# print(to_date)


In [102]:
print(type(from_date))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [103]:
# from_ts = pd.to_datetime(from_date)
# to_ts = pd.to_datetime(to_date)
# print(type(from_date))

# bigint_value_from_ts = int(from_ts.timestamp() * 1000)
# bigint_value_to_ts = int(to_ts.timestamp() * 1000)
# print(type(bigint_value_from_ts))


In [107]:
# Convert from_date and to_date to timestamps in milliseconds
from_ts = int(from_date.timestamp() * 1000)
to_ts = int(to_date.timestamp() * 1000)

# Create a query to join the predictions and actuals feature groups by `pickup_hour` and `pickup_location_id`
query = predictions_fg.select_all() \
    .join(actuals_fg.select(['pickup_location_id', 'pickup_hour']),
            on=['pickup_hour', 'pickup_location_id'], prefix='actuals_') \
    .filter(predictions_fg.pickup_hour >= from_ts) \
    .filter(predictions_fg.pickup_hour <= to_ts)


In [108]:
query

In [109]:
from src.logger import get_logger
logger = get_logger()
from src.feature_store_api import get_or_create_feature_group, get_feature_store
# create the feature view `config.FEATURE_VIEW_MONITORING` if it does not
# exist yet
feature_store = get_feature_store()
# try:
# create feature view as it does not exist yet
feature_store.create_feature_view(
    name=config.MONITORING_FV_NAME, # monitoring_feature_view
    version=5,
    query=query
    )
# except:
#     logger.info('Feature view already existed. Skip creation.')

2025-01-25 18:26:11,269 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-25 18:26:11,273 INFO: Initializing external client
2025-01-25 18:26:11,273 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-25 18:26:12,502 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1192104/featurestores/1181777/featureview). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270244,"usrMsg":"pickup_hour is not primary key in feature group: time_series_hourly_feature_group","errorMsg":"foreign key from the left feature group is not a primary key in the right feature group"}', error code: 270244, error msg: foreign key from the left feature group is not a primary key in the right feature group, user msg: pickup_hour is not primary key in feature group: time_series_hourly_feature_group

In [45]:
# feature view
monitoring_fv = feature_store.get_feature_view(
    name=config.MONITORING_FV_NAME,
    version=5,
)

In [46]:
monitoring_fv.schema

[Training Dataset Feature('pickup_location_id', 'int', 0, False, pickup_location_id, 1393354, None),
 Training Dataset Feature('predicted_demand', 'double', 1, False, predicted_demand, 1393354, None),
 Training Dataset Feature('pickup_hour', 'timestamp', 2, False, pickup_hour, 1393354, None),
 Training Dataset Feature('pickup_ts', 'bigint', 3, False, pickup_ts, 1393354, None),
 Training Dataset Feature('actualspickup_location_id', 'int', 4, False, pickup_location_id, 1393062, None),
 Training Dataset Feature('actualspickup_ts', 'bigint', 5, False, pickup_ts, 1393062, None),
 Training Dataset Feature('actualsrides', 'bigint', 6, False, rides, 1393062, None)]

In [51]:
# fetch data form the feature view
# fetch predicted and actual values for the last 30 days
monitoring_df = monitoring_fv.get_batch_data(
    start_time=from_date - timedelta(days=7),
    end_time=to_date + timedelta(days=7),
)
monitoring_df

2025-01-19 00:06:11,316 ERROR: Binder Error: Cannot compare values of type TIMESTAMP WITH TIME ZONE and type BIGINT - an explicit cast is required
LINE 3: ...fg0"."pickup_location_id" "actualspickup_location_id", "fg0"."pickup_ts" "actualspickup_ts", "fg0"."rides" "actualsrides"
FROM "nyc_taxi_demand_project.model_predictions_feature_group_1" "fg1"
ASOF LEFT JOIN "nyc_taxi_demand_project.time_series_hourly_feature_group_3" "fg0" ON "fg1"."pickup_ts" = "fg0"."pickup_ts" AND "fg1"."pickup_location_id" = "fg0"."pickup_location_id" AND "fg1"."pickup_hour" >= "fg0"."pickup_ts"
                                                  ^. Detail: Python exception: FlyingDuckException. gRPC client debug context: UNKNOWN:Error received from peer ipv4:51.79.26.27:5005 {grpc_message:"Binder Error: Cannot compare values of type TIMESTAMP WITH TIME ZONE and type BIGINT - an explicit cast is required\nLINE 3: ...fg0\".\"pickup_location_id\" \"actualspickup_location_id\", \"fg0\".\"pickup_ts\" \"actualspicku

FeatureStoreException: Could not read data using Hopsworks Feature Query Service.

In [49]:


# # filter data to the time period we are interested in
# pickup_ts_from = int(from_date.timestamp() * 1000)
# pickup_ts_to = int(to_date.timestamp() * 1000)

# add column with Unix epoch milliseconds
pickup_ts_from = from_date.astype(int) // 10**6
pickup_ts_to  = to_date.astype(int) // 10**6


monitoring_df = monitoring_df[monitoring_df.pickup_ts.between(pickup_ts_from, pickup_ts_to)]



2025-01-19 00:05:53,581 ERROR: Binder Error: Cannot compare values of type TIMESTAMP WITH TIME ZONE and type BIGINT - an explicit cast is required
LINE 3: ...fg0"."pickup_location_id" "actualspickup_location_id", "fg0"."pickup_ts" "actualspickup_ts", "fg0"."rides" "actualsrides"
FROM "nyc_taxi_demand_project.model_predictions_feature_group_1" "fg1"
ASOF LEFT JOIN "nyc_taxi_demand_project.time_series_hourly_feature_group_3" "fg0" ON "fg1"."pickup_location_id" = "fg0"."pickup_location_id" AND "fg1"."pickup_ts" = "fg0"."pickup_ts" AND "fg1"."pickup_hour" >= "fg0"."pickup_ts"
                                                  ^. Detail: Python exception: FlyingDuckException. gRPC client debug context: UNKNOWN:Error received from peer ipv4:51.79.26.27:5005 {created_time:"2025-01-19T00:05:53.581294+00:00", grpc_status:2, grpc_message:"Binder Error: Cannot compare values of type TIMESTAMP WITH TIME ZONE and type BIGINT - an explicit cast is required\nLINE 3: ...fg0\".\"pickup_location_id\" \"a

FeatureStoreException: Could not read data using Hopsworks Feature Query Service.

In [5]:
comparison_df = predictions_df.merge(
    actuals_df,
    on=['pickup_hour', 'pickup_location_id'],
    how='left',
    suffixes=('_pred', '_actual')
)
comparison_df

# # Add comparison metrics
comparison_df['prediction_error'] = comparison_df['predicted_demand'] - comparison_df['rides']

# # View aligned predictions and actuals
print(comparison_df[['pickup_hour', 'pickup_location_id', 'predicted_demand', 'rides', 'prediction_error']])

                  pickup_hour  pickup_location_id  predicted_demand  rides  \
0   2024-10-30 23:00:00+00:00                   8              11.0      0   
1   2024-10-30 23:00:00+00:00                  67              13.0      0   
2   2024-10-30 23:00:00+00:00                 250              11.0      0   
3   2024-10-30 23:00:00+00:00                  91              12.0      0   
4   2024-10-30 23:00:00+00:00                 265              10.0      0   
..                        ...                 ...               ...    ...   
250 2024-10-30 23:00:00+00:00                 197              12.0      0   
251 2024-10-30 23:00:00+00:00                  42               9.0      1   
252 2024-10-30 23:00:00+00:00                  14              12.0      1   
253 2024-10-30 23:00:00+00:00                  54              11.0      0   
254 2024-10-30 23:00:00+00:00                  28              12.0      0   

     prediction_error  
0                11.0  
1              